#YOLOv4 Drone detection

In [ ]:
import os
import shutil
import time
from google.colab import drive
from glob import glob
import numpy as np
import pandas as pd

In [ ]:
!nvidia-smi

In [ ]:
drive.mount("/content/drive")

##Cloning repositories

In [ ]:
!git clone https://github.com/tjuric03/DroneDetection
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

##Installing dependencies

In [ ]:
!apt install libopencv-dev python-opencv ffmpeg

##Building darknet

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

!make

##Downloading weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

#Training YOLOv4-tiny model

## Creating training directory

In [ ]:
TRAINING_TIME = time.strftime("%Y%m%d-%H%M%S")

TRAINING_DIR_PATH = "training_" + TRAINING_TIME
os.mkdir(TRAINING_DIR_PATH)

## Generate .cfg file

In [ ]:
CFG_MODEL_NAME = "yolov4-tiny-4class.cfg"

TINY_WEIGHTS_PATH = "yolov4-tiny.weights"
BASE_CONFIG_PATH = "cfg/yolov4-tiny-custom.cfg"
TINY_WEIGHTS_29_PATH = f"{TRAINING_DIR_PATH}/yolov4-tiny.conv.29"
TRAINING_CONFIG_PATH = f"{TRAINING_DIR_PATH}/{CFG_MODEL_NAME}"

In [ ]:
!./darknet partial {BASE_CONFIG_PATH} {TINY_WEIGHTS_PATH} {TINY_WEIGHTS_29_PATH} 29

In [ ]:
!cp {BASE_CONFIG_PATH} {TRAINING_CONFIG_PATH}

In [ ]:
NUMBER_OF_CLASSES = 4
BATCH_SIZE = 64
SUBDIVISIONS = 4
MAX_BATCHES = max(6000, NUMBER_OF_CLASSES * 2000)
STEP1 =  int(0.8 * MAX_BATCHES)
STEP2 = int(0.9 * MAX_BATCHES)
WIDTH = 416
HEIGHT = 416
RANDOM = 1


!sed -i 's/^classes=.*/classes={NUMBER_OF_CLASSES}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^batch=.*/batch={BATCH_SIZE}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^subdivisions=.*/subdivisions={SUBDIVISIONS}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^max_batches.*/max_batches={MAX_BATCHES}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^steps=.*/steps={STEP1},{STEP2}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^width=.*/width={WIDTH}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^height=.*/height={HEIGHT}/g' {TRAINING_CONFIG_PATH}
!sed -i 's/^random=.*/random={RANDOM}/g' {TRAINING_CONFIG_PATH}

#NOTE: MUST CHANGE filters=(classes+5)x3 in each convolutional layer before yolo MANUALLY

## Load config

In [ ]:
LOAD_CONFIG_NAME = "yolov4-tiny-4class.cfg"
CONFIGS_PATH = "/content/drive/MyDrive/training/Configs/"
CONFIG_LOAD_PATH = CONFIGS_PATH + LOAD_CONFIG_NAME

if(os.path.exists(CONFIG_LOAD_PATH)):
  print(f"Replacing config at path {TRAINING_CONFIG_PATH}")
  shutil.copyfile(CONFIG_LOAD_PATH,TRAINING_CONFIG_PATH)
else:
  print(f"No config found at path {CONFIG_LOAD_PATH}")

In [ ]:
with open(f"{TRAINING_DIR_PATH}/obj.names", "w") as fp:
    fp.write("""drone
bird
plane
heli""")

DRIVE_TRAINING_PATH = "/content/drive/MyDrive/training/"+TRAINING_DIR_PATH
if(not os.path.exists(DRIVE_TRAINING_PATH)):
  os.mkdir(DRIVE_TRAINING_PATH)
else:
  print(f"{DRIVE_TRAINING_PATH} already exists! Not creating it again")

with open(f"{TRAINING_DIR_PATH}/obj.data", "w") as fp:
    fp.write(f"""classes = {NUMBER_OF_CLASSES}
train  = {TRAINING_DIR_PATH}/train.txt
valid  = {TRAINING_DIR_PATH}/test.txt
names = {TRAINING_DIR_PATH}/obj.names
backup = {DRIVE_TRAINING_PATH}""")

In [ ]:
IMAGES_PATH = "../DroneDetection/Data"

### Stratified sampling

In [ ]:
from sklearn.model_selection import train_test_split

images = pd.DataFrame(glob(f"{IMAGES_PATH}/*/*.JPEG"),columns=["image_path"])

images["type"] = images.apply(lambda row: row[0].split("/")[-2],axis=1)

train, test = train_test_split(images,test_size=0.1,stratify=images["type"])

In [ ]:
train

In [ ]:
train["image_path"].to_csv(f"{TRAINING_DIR_PATH}/train.txt",index=False,header=False)
test["image_path"].to_csv(f"{TRAINING_DIR_PATH}/test.txt",index=False,header=False)

## Copy all the information needed for training to Drive

In [ ]:
!cp -r "{TRAINING_DIR_PATH}/." {DRIVE_TRAINING_PATH}

## Train command with logging directly to Drive

In [ ]:
!./darknet detector train "{TRAINING_DIR_PATH}/obj.data" {TRAINING_CONFIG_PATH} {TINY_WEIGHTS_29_PATH} -dont_show -map | tee -a "{DRIVE_TRAINING_PATH}/logs.txt"
!cp chart.png {DRIVE_TRAINING_PATH}

## Resume training

In [ ]:
DRIVE_TRAINING_PATH

In [ ]:
!./darknet detector train "{TRAINING_DIR_PATH}/obj.data" {TRAINING_CONFIG_PATH} "{DRIVE_TRAINING_PATH}/yolov4-tiny-4class_last.weights" -dont_show | tee -a "{DRIVE_TRAINING_PATH}/logs.txt"

## Testing trained YOLOv4-tiny model

In [ ]:
!./darknet detect cfg/yolov4.cfg yolov4.weights -thresh 0.25 data/spg3.jpg

In [ ]:
from google.colab.patches import cv2_imshow
#!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
img = cv2.imread('../DroneDetection/Data/Drone/drone_0.JPEG', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

In [ ]:
chart.png